In [1]:
import nbimporter
import numpy as np
import datetime

In [2]:
import SolarPosision
from SolarPosision import SolarPosision

Importing Jupyter notebook from SolarPosision.ipynb


In [3]:
import Weather

Importing Jupyter notebook from Weather.ipynb


In [4]:
import Schedule

Importing Jupyter notebook from Schedule.ipynb
Importing Jupyter notebook from ACSet.ipynb
Importing Jupyter notebook from AnnualCal.ipynb
Importing Jupyter notebook from mdlLibrary.ipynb
Importing Jupyter notebook from Appl.ipynb
Importing Jupyter notebook from Light.ipynb
Importing Jupyter notebook from LocalVent.ipynb
Importing Jupyter notebook from Resi.ipynb


In [5]:
import WallMng
from WallMng import WallMng

Importing Jupyter notebook from WallMng.ipynb
Importing Jupyter notebook from Wall.ipynb


In [6]:
import ExsrfMng
from ExsrfMng import ExsrfMng

Importing Jupyter notebook from ExsrfMng.ipynb
Importing Jupyter notebook from Exsrf.ipynb


In [7]:
import WindowMng
from WindowMng import WindowMng

Importing Jupyter notebook from WindowMng.ipynb


In [8]:
import Sunbrk
from Sunbrk import SunbrkMng

Importing Jupyter notebook from Sunbrk.ipynb


In [9]:
import SeasonalValue
from SeasonalValue import SeasonalValue

Importing Jupyter notebook from SeasonalValue.ipynb


# 室温・熱負荷を計算するクラス

## １）室内部位に関連するクラス

室内部位に関する情報を保持します。

- skin:      外皮フラグ, 外皮の場合True
- boundary:  方位・隣室名, string
- unsteady:  非定常フラグ, 非定常壁体の場合True
- name:      壁体・開口部名称, string
- area:      面積, m2
- sunbreak:  ひさし名称, string
- flr:       放射暖房吸収比率, －
- fot:       人体に対する形態係数, －

In [10]:
# 室内部位に関連するクラス
class Surface:
    
    # 初期化
    def __init__(self, WallMng, WindowMng, skin, boundary, unsteady, name, area, sunbreak, flr, fot):
        self.__skin = skin            #外皮フラグ
        self.__boundary = boundary    #方位・隣室名
        
        # 外皮の場合は方位クラスを取得する
        if self.__skin == True :
            self.__objExsrf = ExsrfMng.ExsrfobjByName(self.__boundary)
        
        self.__unsteady = unsteady    #非定常フラグ
        self.__name = name            #壁体名称
        
        if '床' in self.__name :      #床フラグ
            self.__floor = True
        else:
            self.__floor = False
            
        self.__area = area            #面積
        self.__sunbreak = sunbreak    #ひさし名称
        self.__flr = flr              #放射暖房吸収比率
        self.__for = fot              #人体に対する形態係数
        #self.__floor = floor          #床フラグ
        
        #形態係数収録用リストの定義
        self.__FF = []
        
        #print(self.__unsteady)
        # 壁体の初期化
        if self.__unsteady == True :
            #print(self.__name)
            
            self.__Row = WallMng.Row(self.__name)        #公比の取得
            self.__Nroot = WallMng.Nroot(self.__name)    #根の数
            self.__RFT0 = WallMng.RFT0(self.__name)      #貫流応答の初項
            self.__RFA0 = WallMng.RFA0(self.__name)      #吸熱応答の初項
            self.__RFT1 = WallMng.RFT1(self.__name)      #指数項別貫流応答の初項
            self.__RFA1 = WallMng.RFA1(self.__name)      #指数項別吸熱応答の初項
            self.__oldTsd_a = []
            self.__oldTsd_t = []
            self.__oldTsd_t = range(0, self.__Nroot-1)   #貫流応答の表面温度の履歴
            self.__oldTsd_a = range(0, self.__Nroot-1)   #吸熱応答の表面温度の履歴
            self.__hi = WallMng.hi(self.__name)          #室内側表面総合熱伝達率
            self.__hic = WallMng.hic(self.__name)        #室内側表面対流熱伝達率
            self.__hir = WallMng.hir(self.__name)        #室内側表面放射熱伝達率
            self.__ho = WallMng.ho(self.__name)          #室外側表面総合熱伝達率
            self.__as = WallMng.Solas(self.__name)       #室側側日射吸収率
            self.__Eo = WallMng.Eo(self.__name)          #室内側表面総合熱伝達率
            self.__oldTeo = 15.0                         #前時刻の室外側温度
            self.__olddqi = 0.0                          #前時刻の室内側表面熱流
        # 定常部位の初期化
        else:
            objWindow = WindowMng.Window(self.__name)    #Windowオブジェクトの取得
            self.__tau = objWindow.T()        #日射透過率
            self.__B = objWindow.B()          #吸収日射取得率
            self.__Uso = objWindow.Uso()      #熱貫流率（表面熱伝達抵抗除く）
            self.__RFA0 = 1.0 / self.__Uso                 #吸熱応答係数の初項
            self.__RFT0 = 1.0                              #貫流応答係数の初項
            self.__hi = objWindow.hi()        #室内側表面総合熱伝達率
            self.__hic = objWindow.hic()      #室内側表面対流熱伝達率
            self.__hir = objWindow.hir()      #室内側表面放射熱伝達率
            self.__ho = objWindow.ho()        #室外側表面総合熱伝達率
            self.__U = 1.0 / (1.0 / self.__Uso + 1.0 / self.__hi)   #熱貫流率（表面熱伝達抵抗含む）
            self.__Eo = objWindow.Eo()        #室内側表面総合熱伝達率
            self.__sunbrkflg = len(self.__sunbreak) > 0  #庇がついているかのフラグ
    
    #床フラグの取得
    def Floor(self):
        return self.__floor
    
    #部位面積の取得
    def area(self):
        return self.__area
    
    #応答係数の初項の取得
    def RFA0(self):
        return self.__RFA0
    def RFT0(self):
        return self.__RFT0
    
    #指数項別応答係数
    def RFA1(self):
        return self.__RFA1
    def RFT1(self):
        return self.__RFT1
    
    #公比の取得
    def Row(self):
        if self.__unsteady == True:
            return self.__Row
        else:
            return 0
    
    #室内側表面総合熱伝達率
    def hi(self):
        return self.__hi
    
    #放射暖房吸収比率の取得
    def flr(self):
        return self.__flr
    
    #放射熱伝達率の取得
    def hir(self):
        return self.__hir
    
    #形態係数収録用メモリの確保
    #def FF_alloc(self, Nsurf):
    #    self.__FF = [0:Nsurf-1]
    #    #self.__FF = range(0, Nsurf-1)
    #    print('FFの要素数=', len(self.__FF))
    
    #形態係数の設定
    def setFF(self, FFd):
        self.__FF.append(FFd)
    #形態係数の取得
    def FF(self, i):
        return self.__FF[i]

## ２）室間換気量に関するクラス

室間換気量に関する情報を保持します。

- roomname:      流入室名称
- winter:        暖房期の流入量, m3/h
- inter:         中間期の流入量, m3/h
- summer:        冷房期の流入量, m3/h

In [11]:
class NextVent:
    def __init__(self, roomname, winter, inter, summer):
        self.__roomname = roomname
        self.__SeasonalValue = SeasonalValue(winter, inter, summer)
#        self.__winter = winter                     #暖房期の流入量
#        self.__inter = inter                       #中間期の流入量
#        self.__summer = summer                     #冷房期の流入量
    
    #季節名称から風量を返す
    def next_vent(self, strseason) :
        if strseason == '暖房':
            return self.__SeasonalVaue.winter
        elif strseason == '中間':
            return self.__SeasonalVaue.inter
        elif strseason == '冷房':
            return self.__SeasonalVaue.summer
        else:
            print('<next_vent> not defined season =', strseason)

## ４）空間に関するクラス

空間に関する情報を保持します。

- roomname:      室区分, string
- roomdiv:       室名称, string
- HeatCcap:      最大暖房能力（対流）[W]
- HeatRcap:      最大暖房能力（放射）[W]
- CoolCcap:      最大冷房能力（対流）[W]
- CoolRcap:      最大冷房能力（放射）[W]
- Vol:           室気積, m3
- Fnt:           家具熱容量, kJ/m3K
- Vent:          計画換気量, m3/h
- Inf:           すきま風量[Season]（list型、暖房・中間期・冷房の順）, m3/h
- CrossVentRoom: 通風計算対象室, False
- RadHeat:       放射暖房対象室フラグ, True
- Betat:         放射暖房対流比率, －
- RoomtoRoomVents:      室間換気量（list型、暖房・中間期・冷房、風上室名称）, m3/h
- d:             室内部位に関連するクラス, Surface

In [12]:
# 空間に関する情報の保持
class Space:
    FsolFlr = 0.5                                 #床の日射吸収比率
    # 初期化
    def __init__(self, WallMng, WindowMng, roomname, roomdiv, HeatCcap, HeatRcap, \
                 CoolCcap, Vol, Fnt, Vent, Inf, CrossVentRoom, \
                 RadHeat, Beta, RoomtoRoomVents, Surfaces):
        self.__roomname = roomname                #室用途（主たる居室、その他居室、非居室）
        self.__roomdiv = roomdiv                  #室名称（主寝室、子供室1、子供室2、和室）
        self.__AnnualLoadcC = 0.0                 #年間冷房熱負荷（対流成分）
        self.__AnnualLoadcH = 0.0                 #年間暖房熱負荷（対流成分）
        self.__AnnualLoadrC = 0.0                 #年間冷房熱負荷（放射成分）
        self.__AnnualLoadrH = 0.0                 #年間暖房熱負荷（放射成分）
        self.__Tr = 15.0                          #室温の初期化
        self.__oldTr = 15.0                       #前時刻の室温の初期化
        self.__kc = 0.5                           #人体表面の熱伝達率の対流成分比率
        self.__kr = 0.5                           #人体表面の熱伝達率の放射成分比率
        self.__HeatCcap = HeatCcap                #最大暖房能力（対流）
        self.__HeatRcap = HeatRcap                #最大暖房能力（放射）
        self.__CoolCcap = CoolCcap                #最大冷房能力（対流）
        #self.__CoolRcap = CoolRcap                #最大冷房能力（放射）
        self.__Vol = Vol                          #室気積
        self.__Fnt = Fnt                          #家具熱容量
        #print(self.__Vol, self.__Fnt)
        #print(self.__Vol * 1.2 * 1005.0)
        #print(self.__Vol * self.__Fnt * 1000.0)
        #print('aaaaa')
        #print('Hcap=', self.__Hcap)
        # 室空気の熱容量（家具を含む）
        self.__Hcap = self.__Vol * 1.2 * 1005.0 + self.__Vol * self.__Fnt * 1000.0
        #print(self.__Hcap)
        self.__Vent = Vent                        #計画換気量
        #print(self.__Vent)
        self.__Inf = Inf                          #すきま風量
        self.__CrossVentRoom = CrossVentRoom      #通風対象室フラグ
        self.__RadHeat = RadHeat                  #放射暖房設置フラグ
        self.__Beta = Beta                        #放射暖房対流比率
        #self.__oldNextRoom = []
        #self.__NextVent = []
        self.__RoomtoRoomVent = RoomtoRoomVents
        #print(self.__RoomtoRoomVent)
        #for i in range(1, 3):
        #    for j in range(0, len(NextVent)):
        #        self.__NextVent[i][j] = NextVent[i][j]    #室間換気量
        #        self.__oldNextRoom[j] = 15.0              #風上室の前時刻の温度
        self.__Nsurf = 0                           #部位の数
        self.__Surface = []
        #def __init__(self, skin, boundary, unsteady, name, area, sunbreak, flr, fot)
        
        #部位表面
        self.__Surface = Surfaces
        print('self.__Surfaceの型：', type(self.__Surface))
        #print(Surfaces)
        #for d_surface in Surfaces:
        #    print(d_surface['name'])
        #    self.__Surface.append(Surface(WallMng, d_surface['skin'], d_surface['boundary'], d_surface['unsteady'], \
        #                                  d_surface['name'], d_surface['area'], d_surface['sunbreak'], \
        #                                  d_surface['flr'], d_surface['fot']))
        #    self.__Nsurf += 1                         #定義部位数をインクリメント
        #部位表面数
        #self.__Nsurf = 5
        self.__Nsurf = len(self.__Surface)
        print(self.__Nsurf)
        
        #面対面の形態係数の計算
        self.__Atotal = 0.0
        
        #print('合計面積1=', self.__Atotal)
        #合計面積の計算
        for nxtsurface in self.__Surface:
            self.__Atotal += nxtsurface.area()
        
        #print('合計表面積=', self.__Atotal)
        #形態係数の計算（面積比）
        i = 0
        for surface in self.__Surface:
            j = 0
            #形態係数収録用メモリの確保
            #surface.FF_alloc(self.__Nsurf)
            for nxtsurface in self.__Surface:
                surface.setFF(nxtsurface.area() / self.__Atotal)
                #print(i, j, surface.FF(j))
                j += 1
            i += 1
                
        #放射収支計算のための行列準備
        #行列の準備と初期化
        #[AX]
        self.__matAXd = [ [ 0.0 for i in range(self.__Nsurf) ] for j in range(self.__Nsurf) ]
        #{FIA}
        self.__matFIA = [ [ 0.0 for i in range(1) ] for j in range(self.__Nsurf) ]
        #{CRX}
        self.__matCRX = [ [ 0.0 for i in range(1) ] for j in range(self.__Nsurf) ]
        #{CVL}
        self.__matCVL = [ [ 0.0 for i in range(1) ] for j in range(self.__Nsurf) ]
        #{FLB}
        self.__matFLB = [ [ 0.0 for i in range(1) ] for j in range(self.__Nsurf) ]
        #{WSC}
        self.__matWSC = [ [ 0.0 for i in range(1) ] for j in range(self.__Nsurf) ]
        #{WSR}
        #self.__matWSR = [ [ 0.0 for i in range(1) ] for j in range(self.__Nsurf) ]
        #{WSV}
        self.__matWSV = [ [ 0.0 for i in range(1) ] for j in range(self.__Nsurf) ]
        #{WSB}
        #self.__matWSB = [ [ 0.0 for i in range(1) ] for j in range(self.__Nsurf) ]
        
        #print('FIA[len]=', len(self.__matFIA))
        i = 0
        for surface in self.__Surface:
            #matFIAの作成
            #print('RFA0=', surface.RFA0())
            #print('hi=', surface.hi())
            self.__matFIA[i][0] = surface.RFA0() * surface.hi()
            #print('i=', i, 'FIA=', self.__matFIA[i][0])
            #FLB=φA0×flr×(1-Beta)
            self.__matFLB[i][0] = surface.RFA0() * surface.flr() \
                        * (1. - self.__Beta) / surface.area()
            
            #放射計算のマトリックス作成
            for j in range(self.__Nsurf):
                #print('i=', i, 'j=', j)
                #print('FIA=', self.__matFIA[0][i])
                #print('FF=', surface.FF(j))
                #対角要素
                if i == j:
                    self.__matAXd[i][j] = 1. + self.__matFIA[i][0] \
                            - surface.RFA0() * surface.hir() * surface.FF(j)
                #対角要素以外
                else:
                    self.__matAXd[i][j] = - surface.RFA0() * surface.hir() * surface.FF(j)
            #print('放射計算マトリックス作成完了')
            i += 1
        
        print('[Matrix AXd]')
        print(self.__matAXd)
        
        #逆行列の計算
        self.__matAX = np.linalg.inv(self.__matAXd)
        print('[Matrix AX  inverse Matrix AXd]')
        print(self.__matAX)
        #{WSR}の計算
        self.__matWSR = np.dot(self.__matAX, self.__matFIA)
        print('[Matrix FIA]')
        print(self.__matFIA)
        print('[Matrix WSR]')
        print(self.__matWSR)
        #{WSB}の計算
        self.__matWSB = np.dot(self.__matAX, self.__matFLB)
        print('[Matrix FLB]')
        print(self.__matFLB)
        print('[Matrix WSB]')
        print(self.__matWSB)
    
    #室温を返す
    def oldTr(self):
        return self.__oldTr
    
    #室温、熱負荷の計算
    def calcHload(self, SolarPosision, Schedule, Weather):
        #室間換気の風上室温をアップデート
        return 0

### Constructor parameters
```
d = {
    'Space': [
        {
            'roomname'             : # 室区分
            'roomdiv'              : # 室名称
            'HeatCcap'             : # 最大暖房能力（対流）[W]
            'HeatRcap'             : # 最大暖房能力（放射）[W]
            'CoolCcap'             : # 最大冷房能力（対流）[W]
            'CoolRcap'             : # 最大冷房能力（放射）[W]
            'Vol'                  : # 室気積, m3
            'Fnt'                  : # 家具熱容量, kJ/m3K
            'Vent': {              : # 計画換気量, m3/h
                'winter',
                'inter',
                'summer'
            }
            'Inf' : {
                'winter'           : #冬期のすきま風量, m3/h
                'inter'            : #中間期のすきま風量, m3/h
                'summer'           : #夏期のすきま風量, m3/h
            }                      : # すきま風量[Season]（list型、暖房・中間期・冷房の順）, m3/h
            'CrossVentRoom'        : # 通風計算対象室, False
            'RadHeat'              : # 放射暖房対象室フラグ, True
            'Beta'                 : # 放射暖房対流比率, －
            'NextVent' : [         : # 室間換気量, m3/h
                {
                    'name'         : #室名称
                    'winter'       : #冬期のすきま風量, m3/h
                    'inter'        : #中間期のすきま風量, m3/h
                    'summer'       : #夏期のすきま風量, m3/h
                }
            ]
            'Surface' : [          : # 室内部位に関連するクラス, Surface
                {
                    'skin'         : # 外皮フラグ, 外皮の場合True
                    'boundary'     : # 方位・隣室名, string
                    'unsteady'     : # 非定常フラグ, 非定常壁体の場合True
                    'name'         : # 壁体・開口部名称, string
                    'area'         : # 面積, m2
                    'sunbreak'     : # ひさし名称, string
                    'flr'          : # 放射暖房吸収比率, －
                    'fot'          : # 人体に対する形態係数, －
                    'floor'        : # 床フラグ, 床の場合True（日射が多く当たる）
                }
            ]
        }
    ]
}
```

In [13]:
class SpaceMng:
    def __init__(self, WallMng, WindowMng, d):
        #空間定義の配列を作成
        self.__objSpace = []
        #print(d)
        self.SpacedataRead(WallMng, WindowMng, d)
    
    def SpacedataRead(self, WallMng, WindowMng, d):
        for d_space in d:
            #print(d_space)
            #部位の情報を保持するクラスをインスタンス化
            Surfaces = []
            for d_surface in d_space['Surface']:
                #skin, boundary, unsteady, name, area, sunbreak, flr, fot, floor
                surface = Surface(WallMng, WindowMng, d_surface['skin'], d_surface['boundary'], d_surface['unsteady'], \
                                  d_surface['name'], d_surface['area'], d_surface['sunbrk'], \
                                  d_surface['flr'], d_surface['fot'])
                Surfaces.append(surface)
                #print(d_surface['boundary'])
            
            #計画換気の読み込み
            Vent_season = d_space['Vent']
            #Vent_season = []
            #for d_vent in d_scpace['Vent']:
            #    Vent_season.append(d_vent)
            
            #すきま風の読み込み
            Inf_season = d_space['Inf']
            #Inf_season = []
            #for d_inf in d_space['Inf']:
            #    Inf_season.append(d_inf)
            
            #室間換気量の読み込み
            NextVent = []
            for d_nextvent in d_space['NextVent']:
                next_vent = NextVent(d_nextvent['name'], \
                                     d_nextvent['winter'], d_nextvent['inter'], d_nextvent['summer'])
                NextVent.append(next_vent)
            
            #空間情報を保持するクラスをインスタンス化
            #__init__(self, WallMng, WindowMng, roomname, roomdiv, HeatCcap, HeatRcap, \
            #     CoolCcap, Vol, Fnt, Vent, Inf, CrossVentRoom, \
            #     RadHeat, Beta, NextVent, Surface):
            space = Space(WallMng, WindowMng, d_space['roomname'], d_space['roomdiv'], \
                          d_space['HeatCcap'], \
                          d_space['HeatRcap'], d_space['CoolCcap'], d_space['Vol'], \
                          d_space['Fnt'], d_space['Vent'], Inf_season, \
                          d_space['CrossVentRoom'], d_space['RadHeat'], d_space['Beta'], NextVent, Surfaces)
            print(space)
            self.__objSpace.append(space)
        
        print(self.__objSpace)
    #室温、熱負荷の計算
   # def calcHload(dtmDate, objWdata, objSchedule, objWindowMng, objSunbrk, objExsrfMng):
        